In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [ ]:
groceries = pd.read_csv('groceries/groceries - groceries.csv', delimiter=',')

with open('recipe-ingredients-dataset/train.json', 'r') as f:
    txt = f.read()
recipes = pd.DataFrame(json.loads(txt))

In [ ]:
recipes

In [ ]:
#Check ingredients that appear at least 100 times
from collections import Counter

temp_l = recipes["ingredients"]
temp_l = [j for i in temp_l for j in i]
cnt = Counter(temp_l)

ing_value_count = cnt.most_common()
ing_value_count.sort(key= lambda x: x[1], reverse=True)
ing = [(x,y) for x,y in ing_value_count if y>=100]
len(ing)

In [ ]:
#list of ingredients that appear at least 100 times
ing

In [ ]:
#Remove stopwords 
import re

stopwords = 'confectioners cloves ground black all-purpose kosher green large unsalted extra-virgin purple grated dried chopped \
fresh diced minced brown extract dry white yellow boneless skinless breasts breast flat freshly sea dried \
granulated crushed shredded whole halves peeled coarse warm yolks cold flakes whites cooking fat free less sodium \
sliced finely low light cracked spring lean cooked part-skim roasted sliced flank unsweetened sweetened melted fine \
yellow dark smoked toasted crumbles plain evaporated baking firm red 1% low-fat \
reduced-fat italian greek style whole peeled shoulder plain unbleached dry baby and frozen\
button cremini pure reduced steamed nonfat & de medium extra prepared bow-tie pepper salt'

stopwords = stopwords.split(' ')

def remove_stop_words(query):
    querywords = query.split()
    results_word = [word.lower() for word in querywords if word.lower() not in stopwords]
    results = ' '.join(results_word)
    return results


ingredients_filtered = []

for i in recipes["ingredients"]:
    temp_l = []
    for j in i:
        temp_l.append(remove_stop_words(j))
        
    ingredients_filtered.append(temp_l)
    
recipes["ingredients_filtered"] = ingredients_filtered

In [ ]:
recipes

In [ ]:
#remove empty strings from ingredients_filtered
def remove_empty_strings(l):
    l = [i for i in l if i]
    return l

recipes["ingredients_filtered"] = recipes["ingredients_filtered"].apply(remove_empty_strings)

In [ ]:
# Replacing lemma with token
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

# Ritorna il giusto tag per la parola che analizza (se verbo lo tratta da verbo, se nome da nome, avverbio da avverbio)
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
# 1. Init Lemmatizer
lemmatizer = WordNetLemmatizer()
ingredients_filtered_clean = []

for sentence in recipes['ingredients_filtered']:
    ingredients_filtered_clean.append([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in sentence])

recipes["ingredients_filtered_clean"] = ingredients_filtered_clean

In [ ]:
recipes

In [ ]:
#Replacing plural form with singluar and fixing some mispelling errors

def plural_to_sigular(string):
    
    res = string.replace('onions', 'onion').replace('olives', 'olive').replace('almonds', 'almond').replace('ribs', 'rib').replace('thighs', 'thigh').replace('potatoes', 'potato').replace('mushrooms', 'mushroom').replace('legs', 'leg').replace('carrots', 'carrot').replace('wings', 'wing').replace('steaks', 'steak').replace('eggs', 'egg').replace('bourbon whiskey', 'whiskey').replace('yoghurt', 'yogurt').replace('jack daniels', 'whiskey')
    print(fatto)
    return res

ingredients_filtered_clean = []

for i in recipes["ingredients_filtered"]:
        temp_l = []
        for j in i:
            temp_l.append(plural_to_sigular(j))
        
        ingredients_filtered_clean.append(temp_l)

recipes["ingredients_filtered_clean"] = ingredients_filtered_clean

In [ ]:
recipes

In [ ]:
#Check ingredients count after cleaning procedure
temp_l = recipes["ingredients_filtered_clean"]
temp_l = [j for i in temp_l for j in i]
cnt = Counter(temp_l)

ing_value_count = cnt.most_common()
ing_value_count.sort(key= lambda x: x[1], reverse=True)
top_ing_clean = [(x,y) for x,y in ing_value_count if y>=100]
len(top_ing_clean)

In [ ]:
#Creating a list of the top ingredients (keeping all ingredients)
top_ing_clean_l = [x[0] for x in top_ing_clean]

#(keeping only the top 400)
#top_ing_clean_l = top_ing_clean_l[0:400]

In [ ]:
#Filter out the ingredients keeping only the top ingredients

def filter_ingredients(top_ing, ing):
    ing_filtered = [i for i in ing if i in top_ing]
    return ing_filtered

ingredients_more_filtered = []
for i in recipes["ingredients_filtered_clean"]:
    ingredients_more_filtered.append(filter_ingredients(top_ing_clean_l, i))
    
recipes["ingredients_more_filtered"] = ingredients_more_filtered

In [ ]:
recipes

In [ ]:
#after filtering the ingredients, some recipes are now empty, we remove those empty lists:
boolean_filter = recipes["ingredients_more_filtered"].str.len() == 0
recipes = recipes[boolean_filter == False]

In [ ]:
recipes

In [ ]:
#Create Recipe-Ingredients Matrix
recipes_filtered_df = []

for i in recipes["ingredients_more_filtered"]:
    bin_array = []
    for j in top_ing_clean_l:
        if j in i:
            bin_array.append(1)
        else:
            bin_array.append(0)
    recipes_filtered_df.append(bin_array)
    
recipes_filtered_df = pd.DataFrame(recipes_filtered_df, columns=top_ing_clean_l)

In [ ]:
recipes_filtered_df

In [ ]:
#Apply PCA to this dataset
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

pca.fit(recipes_filtered_df)

pca_sample = pca.transform(recipes_filtered_df)

In [ ]:
#Cluster based on K=10
from sklearn.cluster import KMeans
k = 10
kmeans10 = KMeans(n_clusters=k).fit(recipes_filtered_df)

In [ ]:
print("Top terms per cluster:")
order_centroidsK10 = kmeans10.cluster_centers_.argsort()[:, ::-1]
termsK10 = top_ing_clean_l
for i in range(k):
    top_wordsK10 = [termsK10[ind] for ind in order_centroidsK10[i, :30]]
    print("Cluster {}: {}".format(i, ', '.join(top_wordsK10)))

In [ ]:
import mglearn

#Plot k-means 10 clusters on PCA
plt.figure(figsize=(13,10))
mglearn.discrete_scatter(pca_sample[:,0], pca_sample[:,1], kmeans10.labels_, alpha = 0.5)
plt.legend(['cluster0', 'cluster1', 'cluster2', 'cluster3', 'cluster4',
       'cluster5', 'cluster6', 'cluster7', 'cluster8', 'cluster9'], bbox_to_anchor=(1.02, 1), loc=2)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("K-means (10) on PCA")
plt.show

In [ ]:
#Cluster based on K=50
from sklearn.cluster import KMeans
k = 50
kmeans50 = KMeans(n_clusters=k).fit(recipes_filtered_df)

In [ ]:
print("Top terms per cluster:")
order_centroidsK50 = kmeans50.cluster_centers_.argsort()[:, ::-1]
termsK50 = top_ing_clean_l
for i in range(k):
    top_wordsK50 = [termsK50[ind] for ind in order_centroidsK50[i, :30]]
    print("/n Cluster {}: {}".format(i, ', '.join(top_wordsK50)))

In [ ]:
#Plot k-means 50 clusters on PCA
plt.figure(figsize=(13,10))
mglearn.discrete_scatter(pca_sample[:,0], pca_sample[:,1], kmeans50.labels_, alpha = 0.5)
plt.legend(['cluster0', 'cluster1', 'cluster2', 'cluster3', 'cluster4',
       'cluster5', 'cluster6', 'cluster7', 'cluster8', 'cluster9', 'cluster10',
       'cluster11', 'cluster12', 'cluster13', 'cluster14', 'cluster15',
       'cluster16', 'cluster17', 'cluster18', 'cluster19', 'cluster20', 'cluster21', 'cluster22', 'cluster23', 'cluster24',
       'cluster25', 'cluster26', 'cluster27', 'cluster28', 'cluster29', 'cluster30',
       'cluster31', 'cluster32', 'cluster33', 'cluster34', 'cluster35',
       'cluster36', 'cluster37', 'cluster38', 'cluster39', 'cluster40', 'cluster41', 'cluster42', 'cluster43', 'cluster44',
       'cluster45', 'cluster46', 'cluster47', 'cluster48', 'cluster49'], bbox_to_anchor=(1.02, 1), loc=2)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("K-means (20) cluster on PCA")
plt.show

In [ ]:
#Plot cuisines on PCA
plt.figure(figsize=(13,10))
mglearn.discrete_scatter(pca_sample[:,0], pca_sample[:,1], recipes.cuisine, alpha = 0.5)
plt.legend(['greek', 'southern_us', 'filipino', 'indian', 'jamaican',
       'spanish', 'italian', 'mexican', 'chinese', 'british', 'thai',
       'vietnamese', 'cajun_creole', 'brazilian', 'french', 'japanese',
       'irish', 'korean', 'moroccan', 'russian'], bbox_to_anchor=(1.02, 1), loc=2)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Cuisines on PCA")
plt.show

## Data preparation for market basekt dataset

In [ ]:
# Convert Market Basket dataset to arrays
groceries_array = groceries.values

# Remove NaN from baskets_arrays
groceries_array_clean = []
for i in groceries_array:
   groceries_array_clean.append([x for x in i if str(x) !='nan'])

df_groceries_clean = pd.DataFrame()
df_groceries_clean["items"] = groceries_array_clean

#remove first element from list (the number of elements)
for index, row in df_groceries_clean.iterrows():
    row['items'].pop(0)

In [ ]:
df_groceries_clean.head()

In [ ]:
#Check unique elements in Market basekt dataset
baskets = list(df_groceries_clean["items"])
unique_ele = set(x for l in baskets for x in l)
len(unique_ele)

In [ ]:
#impostare lo stesso formato di dataset che abbiamo per le ricette
#Remove stopwords 
import re

stopwords = 'confectioners cloves ground black all-purpose kosher green large unsalted extra-virgin purple grated dried chopped \
fresh diced minced brown extract dry white yellow boneless skinless breasts breast flat freshly sea dried \
granulated crushed shredded whole halves peeled coarse warm yolks cold flakes whites cooking fat free less sodium \
sliced finely low light cracked spring lean citrus cooked part-skim roasted sliced flank unsweetened sweetened melted fine \
yellow dark smoked boiling toasted crumbles plain evaporated baking firm red 1% low-fat cake \
button cremini pure reduced steamed nonfat & de medium specialty white whole spread root other semi-finished bottled \
canned frozen "(appetizer)" packaged instant misc. '

stopwords = stopwords.split(' ')

def remove_stop_words(query):
    querywords = query.split()
    results_word = [word.lower() for word in querywords if word.lower() not in stopwords]
    results = ' '.join(results_word)
    return results


ingredients_filtered = []

for i in df_groceries_clean["items"]:
    temp_l = []
    for j in i:
        temp_l.append(remove_stop_words(j))
        
    ingredients_filtered.append(temp_l)
    
df_groceries_clean["items_filtered"] = ingredients_filtered

In [ ]:
#remove empty strings from ingredients_filtered
def remove_empty_strings(l):
    l = [i for i in l if i]
    return l

df_groceries_clean["items_filtered"] = df_groceries_clean["items_filtered"].apply(remove_empty_strings)

In [ ]:
df_groceries_clean

In [ ]:
#Check unique elements in Market basekt dataset
baskets = list(df_groceries_clean["items_filtered"])
unique_mb = set(x for l in baskets for x in l)
len(unique_mb)

In [ ]:
# Replacing lemma with token
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

# Ritorna il giusto tag per la parola che analizza (se verbo lo tratta da verbo, se nome da nome, avverbio da avverbio)
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
# Replacing lemma with token
lemmatizer = WordNetLemmatizer()
items_filtered_clean = []

for sentence in df_groceries_clean['items_filtered']:
    items_filtered_clean.append([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in sentence])

df_groceries_clean["items_filtered_clean"] = items_filtered_clean

In [ ]:
df_groceries_clean

In [ ]:
#Check unique elements in recipes dataset
baskets = list(recipes["ingredients_more_filtered"])
unique_recipes = set(x for l in baskets for x in l)
len(unique_recipes)


In [ ]:
# controllo gli elementi in comune tra i due dataset puliti
common_items =[]
for items in unique_recipes:
    if items in unique_mb:
        common_items.append(items)

len(common_items)

## Converto vegetali e frutta in recipes

In [ ]:
import WikipediaExtraction as we

fruits = we.getFruits()
vegetables = we.getVegetables()

In [ ]:
for recipe in recipes["ingredients_more_filtered"]:
    for index, ingredient in enumerate(recipe):
        if(ingredient in fruits):
            recipe[index] = "fruits"
        if("fruit" in ingredient or "fruits" in ingredient):
            recipe[index] = "fruits"
        if(ingredient in vegetables):
            recipe[index] = "vegetables"
        if("vegetable" in ingredient or "vegetables" in ingredient):
            recipe[index] = "vegetables"

In [ ]:
#Check unique elements in recipes dataset
baskets = list(recipes["ingredients_more_filtered"])
unique_recipes = set(x for l in baskets for x in l)
len(unique_recipes)

Notiamo che abbiamo eliminato 40 elementi unici all'interno del dataset recipes

In [ ]:
#Check unique elements in recipes dataset
baskets = list(recipes["ingredients_more_filtered"])
unique_recipes = set(x for l in baskets for x in l)
len(unique_recipes)

# Converto vegetali e frutta su groceries

In [ ]:
for recipe in df_groceries_clean["items_filtered_clean"]:
    for index, ingredient in enumerate(recipe):
        #print("Ingrediente prima: {0}".format(ingredient))
        if(ingredient in fruits):
            recipe[index] = "fruits"
        if("fruit" in ingredient or "fruits" in ingredient):
            recipe[index] = "fruits"
            #print("Frutta: {0}".format(ingredient))
        if(ingredient in vegetables):
            recipe[index] = "vegetables"
        if("vegetable" in ingredient or "vegetables" in ingredient):
            recipe[index] = "vegetables"
            #print("Vegetale: {0}".format(ingredient))

In [ ]:
#Check unique elements in Market basekt dataset
baskets = list(df_groceries_clean["items_filtered_clean"])
unique_mb = set(x for l in baskets for x in l)
len(unique_mb)

In [ ]:
# controllo gli elementi in comune tra i due dataset puliti
common_items =[]
for items in unique_recipes:
    if items in unique_mb:
        common_items.append(items)

len(common_items)

In [ ]:
common_items